<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Acknowledgments" data-toc-modified-id="Acknowledgments-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Acknowledgments</a></span></li><li><span><a href="#Installations,-Imports-and-Settings" data-toc-modified-id="Installations,-Imports-and-Settings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Installations, Imports and Settings</a></span><ul class="toc-item"><li><span><a href="#Installation" data-toc-modified-id="Installation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Installation</a></span></li><li><span><a href="#Imports-and-Settings" data-toc-modified-id="Imports-and-Settings-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Imports and Settings</a></span></li></ul></li><li><span><a href="#Scraping" data-toc-modified-id="Scraping-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scraping</a></span><ul class="toc-item"><li><span><a href="#Page-Source-Scraping" data-toc-modified-id="Page-Source-Scraping-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Page Source Scraping</a></span></li><li><span><a href="#Attraction-Parsing" data-toc-modified-id="Attraction-Parsing-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Attraction Parsing</a></span></li><li><span><a href="#Attraction's-Information-Scraping" data-toc-modified-id="Attraction's-Information-Scraping-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Attraction's Information Scraping</a></span></li></ul></li><li><span><a href="#Data-Collection-and-Preprocessing" data-toc-modified-id="Data-Collection-and-Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Collection and Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Collection" data-toc-modified-id="Collection-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Collection</a></span></li><li><span><a href="#PreProcessing" data-toc-modified-id="PreProcessing-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>PreProcessing</a></span><ul class="toc-item"><li><span><a href="#Duplicates" data-toc-modified-id="Duplicates-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Duplicates</a></span></li><li><span><a href="#Outliers" data-toc-modified-id="Outliers-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Outliers</a></span></li><li><span><a href="#Feature-Selection" data-toc-modified-id="Feature-Selection-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Feature Selection</a></span></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Missing Values</a></span></li></ul></li></ul></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#DBSCAN-in-short" data-toc-modified-id="DBSCAN-in-short-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><code>DBSCAN</code> in short</a></span></li><li><span><a href="#Parameter-Setting" data-toc-modified-id="Parameter-Setting-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Parameter Setting</a></span></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Model-Evaluation" data-toc-modified-id="Model-Evaluation-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Model Evaluation</a></span></li></ul></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusions</a></span></li></ul></div>

# Chicago's Most Visited Attractions

The aim of this notebook is to perform a ***clustering*** exercise. 

Specifically, we will pit the geospatial clusters detected by `DBSCAN` based on taxi trip destinations within the Chicago against the city's most visited attractions, according to TripAdvisor.

The major data science concepts touched upon in this notebook will be:

+ Web Scraping;
+ Data Collection and Preprocessing;
+ Clustering;
+ Geospatial Data.


## Acknowledgments

The ideas bolstering the clustering section are largely based on Professor [Oriol Pujol](http://www.maia.ub.es/~oriol/)'s lecture on Density-based clustering. 

## Installations, Imports and Settings

### Installation

Besides the "classic" libraries, the below libraries are required in order to be able to run this notebook:

`pip install gmaps` (mandatory)

a Jupyter plugin for embedding Google maps in Jupyter notebooks. It is designed to help visualize and interact with geographical data.

`pip install sodapy` (optional if loading the data directly from the `.csv`)

a Python client for the Socrata Open Data API.

Further, the scraping section requires a ***chromedriver.exe*** application. The GitHub repository has one located in folder "ChromeDriver", matching `Chrome version 84`.

In order to work, it is fundamental that the application release and Chrome's version match:

- To check your Chrome's version, consult this guide [here](https://help.zenplanner.com/hc/en-us/articles/204253654-How-to-Find-Your-Internet-Browser-Version-Number-Google-Chrome).

- If you need to utilize another ***chromedriver.exe*** release, the required application can be downloaded from [here](https://chromedriver.chromium.org/downloads). 

### Imports and Settings

In [1]:
# general
import numpy as np
import pandas as pd
from random import randint
import pickle
import json

# scraping
from scraping import tripadvisor_scraper, landmarks_parser, info_scraper

# data collection
from sodapy import Socrata

# clustering
from sklearn.cluster import DBSCAN

# geospatial plotting
import gmaps

# settings modification
import warnings
warnings.filterwarnings('ignore')

# autoreload changes in custom modules
%load_ext autoreload
%autoreload 2

# change the number of displayed columns
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

## Scraping

In this section the scraping is carried out. We will use two of the most common libraries for web scraping, namely ***BeautifulSoup (BS)*** and ***Selenium (SE)***.

In many instances ***BS*** is a powerful enough tool, however, is not sufficient in our case. By performing only static scraping, ***BS*** fetches web pages from the server without the help of a browser, thereby ignoring JavaScript; in other words, it enables to retrieve exclusively what is displayed on the webpage.

On the other hand, ***SE*** automates web browser interaction from Python offering the opportunity to interact with the website.

In short, we will use ***SE*** to automate the scroll down process until the page bottom to load all the page dynamically the button clicks and then can be scraped by means of ***BS***. Such a process is decomposed into 3 steps:

1. retrieve dynamically the page source of the "Attractions" pages (***SE***);

2. parse the sources to find the required types of attractions (***BS***);

3. scrape latitude&longitude and Wikipedia description of the selected attractions (***SE***).

The functions implemented in this section are stored in the file named `scraping.py` and thus imported from there.

### Page Source Scraping

The function `tripadvisor_scraper()` will enable us to retrieve the page source of the "Attractions" pages, returned as a dictionary. 

The only required argument is the self-explanatory `city name`, whereas the `number_of_pages` and `headless` are optionals. 

The default `number_of_pages` is `None` and it refers to the number of pages to be scraped. In both cases when the number is not provided or the inputted number is larger than the actual number of available pages, the function will assign to it a value equal to the total number of available pages. 

`headless` is a Selenium option and simply decides whether to show the browser performing the scraping or not. The default is set to `True`since using the headless browser - with its lower CPU and memory cost and fewer moving parts to crash or hang - is ideal.

In [3]:
page_source_dict = tripadvisor_scraper(city='Chicago', number_of_pages=3, headless=True)

Page 3 out of 3 has been successfully scraped and parsed

Scraping and parsing of Chicago's Tripadvisor pages have been successfully performed


The following screenshots provide an understanding of the scraping process.

![](images/chicago_selection.png)

![](images/things_to_do.png)

![](images/attractions.png)

Below, the first 2000 characters from one page source taken from the dictionary containing all the page sources.

In [68]:
page_source_dict[1][0:2000]

'<html lang="en"><head><meta http-equiv="content-type" content="text/html; charset=utf-8"><link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico?v2" type="image/x-icon"><link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" color="#000000"><meta name="theme-color" content="#34e0a1"><meta name="format-detection" content="telephone=no"><script async="" type="text/javascript" src="//securepubads.g.doubleclick.net/tag/js/gpt.js"></script><script src="https://connect.facebook.net/en_US/sdk.js?hash=17c9a7a22155f3679c749b7562a56f38&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script id="facebook-jssdk" src="//connect.facebook.net/en_US/sdk.js"></script><script type="text/javascript">window.taRollupsAreAsync = true;</script><link rel="stylesheet" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" crossorigin=""><title>Chicago Attractions | Tripadvisor</title><meta name="keyw

### Attraction Parsing

Now that we have retrieved the needed page sources, these will be parsed in order to retrieve the names of the landmarks.

Keep in my mind that we could also define some specific type of attractions, based on our particular need. For this exercise, we will select only the three types shown in the picture.

![](images/landmark_types.png)

In [10]:
# define the attraction types
landmarks_list = ['Museums', 'Nature & Parks', 'Sights & Landmarks']

`landmarks_dict()` is the function that returns a dictionary with the points of interests found in the page sources.

`page_source` is the only mandatory argument and requires a dictionary with the page sources to be parsed.
On the other hand, `landmarks_list` is optional and accepts as input a list containing the type of landmarks to be searched.

In [12]:
# parse the page source to search landmarks
landmarks_dict = landmarks_parser(page_source=page_source_dict, landmarks_list=landmarks_list)

56 landmarks have been successfully inserted in the dictionary


The landmarks found that match the type(s) provided by the user are returned stored in a dictionary. One example is shown below.

In [7]:
landmarks_dict[1]

{'name': 'The Art Institute of Chicago'}

### Attraction's Information Scraping

Once the desired attraction types have been parsed, we can proceed with the scraping of latitude&longitude of the selected attractions. This step is necessary to plot the landmarks on Chicago's GoogleMap, thus making the comparison with the clusters visually compelling.

Further, the Wikipedia description will be also retrieved. This is not strictly necessary but nonetheless it is a fine detail that will enable the user to identify each of the plotted landmarks.

To this end, we have developed the function `points_of_interest()`. As explained above, this function will search the latitude & longitude as well as the Wikipedia description and attach them to its respective vocabulary key, if found (or "NA" otherwise).

The unique mandatory parameter is `points_of_interest`, that is, a dictionary with the landmarks (as `str`) to be scraped. Similarly to the `number_of_pois` and `headless` follow exaclty the same logic as their counterparts in function `tripadvisor_scraper()`.

In [6]:
points_of_interest = info_scraper(points_of_interest=landmarks_dict, number_of_pois=None, headless=True)

Chrome is already in English or English is not available


Landmark 56 out of 56 has been successfully scraped

Scraping of coordinates and description has been successfully performed


An example is presented below, where both pieces of information are actually available. 

![](images/google_search_bar.png)

![](images/landmarks_gps_wiki.png)

Now, we will drop the landamks with etiher missing latitude & longitude or Wikipedia description. 

In [7]:
points_of_interest_list = []
for i in range(1, len(points_of_interest) + 1):
    if points_of_interest[i]['location'] != "NA":
        points_of_interest_list.append({"name": points_of_interest[i]['name'], "location": points_of_interest[i]['location'],
                                        "description": points_of_interest[i]['description']})

Apparently, we have lost some of the original landmarks. 

In [8]:
len(points_of_interest_list)

47

And this is the result of the cleaning, a nested dictionary.

In [9]:
points_of_interest_list[0:2]

[{'name': 'The Art Institute of Chicago',
  'location': (41.8796, -87.6237),
  'description': "The Art Institute of Chicago in Chicago's Grant Park, founded in 1879, is one of the oldest and largest art museums in the United States. Recognized for its curatorial efforts and popularity among visitors, the museum hosts approximately 1.5 million people annually. "},
 {'name': 'Museum of Science and Industry',
  'location': (41.7906, -87.5831),
  'description': "The Museum of Science and Industry is a science museum located in Chicago, Illinois, in Jackson Park, in the Hyde Park neighborhood between Lake Michigan and The University of Chicago. It is housed in the former Palace of Fine Arts from the 1893 World's Columbian Exposition. "}]

Cheers to us! We have no retrieved all the geospatial data and descriptions needed to plot the landmarks on GoogleMap. 

Let's now plot them on the map to get a practical understanding of what we have accomplished. 

<div class="alert alert-block alert-success">
<b>Tip:</b> Run the below cell only if you have if interested in trying the code out. Else, its result is loaded two cells below as picture. 
</div>

In [13]:
# set the map configurations
figure_layout = {
    'width': '1200px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

# build the map base
map_pois = gmaps.figure(map_type='ROADMAP', layout=figure_layout,
                   zoom_level=11, center=(41.834275, -87.691385))

# retrieve the coordinates for the pins to plot on the map
points_of_interest_locations = [poi['location']
                                for poi in points_of_interest_list]

# generate the infobox template for the pins
info_box_template = """
<dl>
<dt>Name:</dt><dd>{name}</dd>
<dt>Description:</dt><dd>{description}</dd>
</dl>
"""
# add specific info to each specific pin
points_of_interest_info = [info_box_template.format(
    **poi) for poi in points_of_interest_list]

# generate the marker layer maps
marker_layer = gmaps.marker_layer(
    points_of_interest_locations, info_box_content=points_of_interest_info)

# open the json with Census boundaries
with open('./data/boundaries_census_tracts.geojson') as json_file:
    boundaries = json.load(json_file)

# generate the district boundaries on the map
geojson_layer = gmaps.geojson_layer(boundaries)

# add all the layers on the map
map_pois.add_layer(marker_layer)
map_pois.add_layer(geojson_layer)

# plot the map
map_pois

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1200px'))

![](images/chicago_map_census_pois.jpg)

The landmarks have been plotted on the map based on their latitude and longitude. Clicking on one of them would generate a pop-up displaying the name and the description.   

Further, as a fine detail to help visualize better Chicago's city limits, `GeoJSON` data about the city's Census Tracts obtained from [here](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9) has also been plotted.

At this point, having scraped some points of interest and their information, we can set these data aside for a moment and move to next phase, the taxi ride data retrieval and cleaning.

## Data Collection and Preprocessing

### Collection

<div class="alert alert-block alert-success">
<b>Tip:</b> Run this section (<i>"4.1 Collection"</i>) if you have not yet saved the dataset locally.  Else, move to section <i>"4.2 Preprocessing"</i>. 
</div>

The dataset about taxi rides in Chicago is publicly available and retrievable from [here](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew).

There are two potential ways of collecting such data:
+  Downloading the available `.csv` file;
+ Socrata Open Data API (SODA) API.

We opt for SODA given its flexibility. In fact, it provides programmatic access to this dataset including the ability to filter, query, and aggregate data, instead of downloading the full 194 million rows.

For this example, we will retrieve 500.000 taxi rides comprised between the first of January 2017 and the same day in 2019. 

In [ ]:
# Unauthenticated client only works with public data sets.
# Note 'None' in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# retrieve the results from the specified online database
# based on the conditions determined in 'where' argument
results = client.get("wrvz-psew", where="trip_start_timestamp > '2017-01-01T00:00:01.000' AND trip_start_timestamp < '2019-01-01T00:00:01.000'",
                     limit=500000)

The collected rides are then stored into a pandas dataframe and eventually saved as a `.csv` file locally. This will enable to quickly load the data in future occasions. 

In [8]:
# convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# add dataframe to csv file named 'taxi_rides.csv'
df.to_csv('./data/taxi_rides.csv', index=False)

### PreProcessing

As usual, we begin by loading the dataset (in this case, only the first 400.000 arbitrarily) and carry out some initial data exploration to get acquainted with the data.

In [14]:
# define the number of rows to be uplaoded (max 500.000)
row_number = 400000

In [15]:
# load the csv, but only the first 250.000 entries 
df = pd.read_csv('./data/taxi_rides.csv', nrows=row_number)

In [16]:
# print the number of rows and the number of columns
print("The dataset has {} rows and {} columns.".format(df.shape[0], df.shape[1]))

The dataset has 400000 rows and 23 columns.


In [17]:
# show the column type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 23 columns):
trip_id                       400000 non-null object
taxi_id                       399798 non-null object
trip_start_timestamp          400000 non-null object
trip_end_timestamp            399961 non-null object
trip_seconds                  399961 non-null float64
trip_miles                    399999 non-null float64
pickup_census_tract           299939 non-null float64
dropoff_census_tract          297227 non-null float64
pickup_community_area         399986 non-null float64
dropoff_community_area        391164 non-null float64
fare                          400000 non-null float64
tips                          400000 non-null float64
tolls                         399993 non-null float64
extras                        400000 non-null float64
trip_total                    400000 non-null float64
payment_type                  400000 non-null object
company                       

The meaning of the features will be briefly explained as we encounter them throughout the notebook, yet, the interested reader can find their thorough description [here](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew).

Anyways, the dataset contains 23 columns and the majority are of `float` type whereas the rest is `object` - the latter meaning that they are composed of text, or mixed numeric and non-numeric values. This can be assessed also by printing out some 10 randomly selected rows. 

In [18]:
# show 5 random rows 
df.sample(10)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
304169,d22f7769b6fe6662bb80bde2324706a7aef71290,473c164e34e4ce1f5df4c7ada1478d8b9d27b10737351c...,2017-03-03T10:45:00.000,2017-03-03T11:00:00.000,660.0,0.0,1.703128e+10,1.703108e+10,28.0,8.0,9.50,0.00,0.0,1.0,10.50,Cash,Northwest Management LLC,41.879255,-87.642649,"{'type': 'Point', 'coordinates': [-87.64264899...",41.895033,-87.619711,"{'type': 'Point', 'coordinates': [-87.61971067..."
116897,a2680eca75a2d478a1b7d21adbf1da01f28c941b,fc0c7db172c81d1a2ab72cc31b27f4917d40ee8d0a43d6...,2017-02-14T18:45:00.000,2017-02-14T19:00:00.000,720.0,0.0,1.703108e+10,1.703128e+10,8.0,28.0,8.00,1.50,0.0,0.0,9.50,Credit Card,Taxi Affiliation Services,41.892042,-87.631864,"{'type': 'Point', 'coordinates': [-87.63186394...",41.879255,-87.642649,"{'type': 'Point', 'coordinates': [-87.64264899..."
36720,e4c60c1ec3e08760484bf2e3114fa3a9550c82a4,29f6d119e21d61401aa5156346aca14fad3d0fea0dc1c8...,2017-02-19T15:00:00.000,2017-02-19T15:15:00.000,660.0,2.7,NaN,NaN,24.0,8.0,10.50,2.00,0.0,1.0,13.50,Credit Card,Taxi Affiliation Services,41.901207,-87.676356,"{'type': 'Point', 'coordinates': [-87.67635598...",41.899602,-87.633308,"{'type': 'Point', 'coordinates': [-87.63330803..."
113560,6e6bb5cb4cdcc903d5c23d106525b8241d11bb2c,0881b7b36b4aad06e5ee59472465b15a6620637f694aae...,2017-01-22T10:00:00.000,2017-01-22T10:15:00.000,360.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,6.50,1.00,0.0,0.0,7.50,Credit Card,Taxi Affiliation Services,41.900221,-87.629105,"{'type': 'Point', 'coordinates': [-87.62910518...",41.893216,-87.637844,"{'type': 'Point', 'coordinates': [-87.63784420..."
120410,5571e9bbdf8d902f1685164e02537e8bbe89c8e6,634bc0295489ab0b50cebcc10d6d3e0b8ed7dc079e9473...,2017-02-15T12:15:00.000,2017-02-15T12:15:00.000,0.0,0.0,NaN,NaN,76.0,76.0,3.25,0.00,0.0,0.0,3.25,Cash,Choice Taxi Association,41.980264,-87.913625,"{'type': 'Point', 'coordinates': [-87.91362459...",41.980264,-87.913625,"{'type': 'Point', 'coordinates': [-87.91362459..."
291580,93aa21300a554b2be29f71192a1a27a37c49b9cd,effecc1cd82175a1d0bbe1def9ef38f52a9f6f437d1981...,2017-02-16T14:15:00.000,2017-02-16T14:30:00.000,720.0,0.1,1.703184e+10,1.703108e+10,32.0,8.0,9.75,1.00,0.0,0.0,10.75,Credit Card,Taxi Affiliation Services,41.880994,-87.632746,"{'type': 'Point', 'coordinates': [-87.63274648...",41.907520,-87.626659,"{'type': 'Point', 'coordinates': [-87.62665890..."
242012,9d42310824c456b42409a32e972fd0a62756fce8,10cb380e744ed76d4cf3461ca8671683cd392139e0e2af...,2017-02-12T23:00:00.000,2017-02-12T23:30:00.000,1620.0,15.0,NaN,NaN,56.0,39.0,38.75,8.95,0.0,6.0,53.70,Credit Card,Taxi Affiliation Services,41.792592,-87.769615,"{'type': 'Point', 'coordinates': [-87.76961545...",41.808916,-87.596183,"{'type': 'Point', 'coordinates': [-87.59618334..."
370364,b17d0b0d546272a98c4515180a97b46b966cfbb7,cd1574bcd03f25b70cb75eab0dbe94b0b04a60cfe2abb0...,2017-02-27T14:00:00.000,2017-02-27T14:15:00.000,1440.0,17.9,1.703184e+10,1.703198e+10,32.0,76.0,43.75,8.75,0.0,0.0,52.50,Credit Card,Top Cab Affiliation,41.880994,-87.632746,"{'type': 'Point', 'coordinates': [-87.63274648...",41.979071,-87.903040,"{'type': 'Point', 'coordinates': [-87.90303966..."
206939,8a4c75465b2a5076d240060c48708a53b57836a9,c4eb7d8517b858b849d78a9708e2a0f102123f6b09bed1...,2017-03-05T01:30:00.000,2017-03-05T01:45:00.000,840.0,0.3,1.703108e+10,1.703107e+10,8.0,7.0,16.75,0.00,0.0,1.0,17.75,Cash,Taxi Affiliation Services,41.892508,-87.626215,"{'type': 'Point', 'coordinates': [-87.62621490...",41.929273,-87.673807,"{'type': 'Point', 'coordinates': [-87.67380723..."
203713,8fdf621013d62add2ab1e16c8d547d41a4e22260,084b422a6d13f5d6b7be4a235cb6cc68363b39f24536de...,2017-02-12T05:4

This first quick exploration also showcases some missing values (i.e. `NaNs`). However, before handling these we should first take care of:

1. duplicates;
2. outliers (e.g. oddly short trips in terms of time and/or distance);
3. feature selection. 

#### Duplicates

According to Chicago's City Hall, two, or more, trips are duplicates only if they have identical values for ([source](https://digital.chicago.gov/index.php/chicago-taxi-data-released/)):

+ Taxi ID (`trip_id`): A unique identifier for the taxi;
+ Trip Start Timestamps (`trip_start_timestamp`): When the trip started, rounded to the nearest 15 minutes;
+ Trip End Timestamps (`trip_start_timestamp`): When the trip ended, rounded to the nearest 15 minutes
+ Trip Seconds (`trip_seconds`): Time of the trip in seconds;
+ Trip Miles (`trip_miles`): Distance of the trip in miles;
+ Pickup Census Tracts (`pickup_census_tract`): The Census Tract where the trip began;
+ Dropoff Census Tracts (`dropoff_census_tract`): The Census Tract where the trip ended;
+ Pickup  Community Areas (`pickup_community_area`): The Community Area where the trip began;
+ Dropoff Community Areas (`dropoff_community_area`): The Community Area where the trip ended.

To this end, we will employ the `drop_duplicates()` function. 

In [19]:
# define the subset of columns in which to look for duplicates
duplicate_col_list = ['taxi_id', 'trip_start_timestamp', 'trip_end_timestamp', 
                      'trip_seconds', 'trip_miles', 'pickup_census_tract',
                      'dropoff_census_tract', 'pickup_community_area', 'dropoff_community_area']

In [20]:
# drop the duplicates and modify the existing dataframe
df.drop_duplicates(subset=duplicate_col_list, 
                   inplace=True)

In [21]:
# print the number of deleted duplicates
print("{} duplicates have been removed".format(row_number-df.shape[0]))

# print the lenght of the new daatframe
print("The new dataframe has {} rows".format(df.shape[0]))

542 duplicates have been removed
The new dataframe has 399458 rows


Now that duplicates have been detected and removed, we can move to the outliers treatment.

#### Outliers

Chicago's City Hall already manipulates the data in order to exclude implausible values. As per their website:

+ Trip times less than zero or greater than 86,400 seconds are removed;
+ Trip lengths less than zero or greater than 3,500 miles are removed.

The City Hall makes a compelling case about the upper limits. Further, by using a density-based clustering techniques, rare long-distance trips will be treated as outliers by the algorithm and thereby easily identified. 

The lower bound, though, appears to be somehow neglected. The City Hall only canceled negative time values. But what to do with oddly short trips?

A trip of, say, 20 seconds would in fact be rather "suspicious" most of the times (we could speculate, as an example, a client arguing with the cab driver and asking to be dropped off immediately). The same goes for rides of, say, 0.1 miles (or 160 meters). 

To obtain a better idea, let's look at a very robust statistic, the ***median***.

In [22]:
# find the median of the features
df[['trip_seconds', 'trip_miles']].median()

trip_seconds    540.0
trip_miles        0.8
dtype: float64

The median reveals that the average trip time is 9 minutes and length of about 0.8 miles (or 1.3 kilometers).

In this vein, taxi rides with either or both of the following will be discarded:

+ Trip times less than 60 seconds;
+ Trip lengths less than 0.1 miles (160 meters).

Obviously, these are arbitrary choices and as such should be considered.

In [23]:
# modify the dataframe by keeping only trips equal or longer than 60 seconds
df = df.loc[df['trip_seconds'] >= 60]

In [24]:
# modify the dataframe by keeping only trips equal or longer than 0.1 miles
df = df.loc[df['trip_miles'] >= 0.1]

In [25]:
# print the lenght of the new daatframe
print("The new dataframe has {} rows".format(df.shape[0]))

The new dataframe has 283694 rows


#### Feature Selection

To carry out the clustering task, we will only need two features, namely:

+ `dropoff_centroid_latitude`: The **latitude** of the center of the dropoff census tract or the community area if the census tract has been hidden for privacy;

+ `dropoff_centroid_longitude`. The **longitude** of the center of the dropoff census tract or the community area if the census tract has been hidden for privacy.

A copy of the original dataframe is then created, containing only the above-mentioned columns. 

In [26]:
# create a copy of the original dataframe containing only the below columns
df_1 = df[['dropoff_centroid_latitude', 'dropoff_centroid_longitude']].copy()

df_1

,dropoff_centroid_latitude,dropoff_centroid_longitude
0,41.880994,-87.632746
2,41.880994,-87.632746
4,41.914616,-87.631717
5,41.907492,-87.635760
6,42.001571,-87.695013
...,...,...
399993,41.878866,-87.625192
399994,41.909496,-87.630964
399995,41.890922,-87.618868
399998,41.901207,-87.676356


#### Missing Values

We are now ready to handle missing values. Dropping missing values is sub-optimal because removing observations, translates into losing information. And the fact that the value was missing may be informative in itself. However, in the clustering case, we cannot simply imput a value - like the mean - for the missing values, or build a column with an indicator variable flagging the original lack, and let the algorithm figure this out.

Further, in this dataset there is a specific reason behind missing values, due to privacy. In fact:
> Since some Census Tracts have relatively infrequent taxi trips, we show the Census Tracts of a trip only if both the starting and ending Census Tracts had at least three trips in the relevant 15-minute time period. 
Because of this rule, about 1/4 of Census Tracts that would otherwise be shown are blank. (Others are blank because of missing data or falling outside Chicago.)

Therefore, we will opt for dropping the observations lacking `latitude` or `longitude`. Performing this exercise on the whole dataset would have been possible, of course. But given that our task at hand only requires two features, it would have entailed dropping observations due to `NaNs` in columns other than the required ones, thereby producing an unnecessary data loss. 

In [27]:
# find the number of NaNs, if any
df_1.isna().sum()

dropoff_centroid_latitude     5797
dropoff_centroid_longitude    5797
dtype: int64

Another interesting aspect we should investigate is whether the missing values are actually coupled.

In [28]:
len(df_1[(df_1['dropoff_centroid_latitude'].isna()) & (df_1['dropoff_centroid_longitude'].isna())])

5797

In fact, we obtain the same number of observations as before, even when we specifically declare both conditions simultaneously.

We now proceed with the exclusions of `NaNs`.

In [29]:
# drop the rows with NaNs and modifies the df1
df_1.dropna(inplace=True)

In [30]:
# print the lenght of the new daatframe
print("The new dataframe has {} rows".format(df_1.shape[0]))

The new dataframe has 277897 rows


## Clustering

### `DBSCAN` in short

`DBSCAN` is able to find arbitrary shaped clusters and clusters with noise (i.e. outliers). The core concept bolstering the algorithm is that a point belongs to a certain cluster if it is arbitrarily close to `x` points from that specific cluster.

In order to run `DBSCAN`, two important parameters need to be adjusted:

- `epsilon`:
    - the distance that specifies the neighborhoods. Two points are considered to be neighbors if the distance between them are less than or equal to it;
- `MinPoints`:
    - minimum number of data points to define a cluster.

These are necessary to define the three different types of points contemplated by the algorithm:

1. *Core points*:
    - A point that has `MinPoints` around itself, within a radius of `epsilon`. These points are clustered together; 
2. *Border points*:
    - A point that is in the range of a core point but has not `MinPoints` points around its `epsilon` range. Normally, border points are located at the edge of the clusters. 
3. *Outliers*:
    - A point that is not in the range of a core point and thus is clustered alone.

Technically, the algorithm works as follows.

1. A starting point is selected at random at its neighborhood area is determined by means of `epsilon`. If there are at least `MinPoints` within the neighborhood, the point is marked as *core* and a cluster formation starts. If not, the point is marked as an *outlier*. 

2. Once said cluster is created - say, cluster A - all the points within the neighborhood of the starting point become part of cluster A. If these new points also happen to be *core*, the points within their neighborhood are also aggregated to cluster A.
Obviously, nothing prevents a point previously marked as *outlier* to be revisited, and thus become part of a cluster.

3. This process continues until no other data points are within the neighborhood formed by `epsilon`. At this point, the algorithm will seek to form another cluster by selecting a not-yet-visited point and applying the above-mentioned procedure again.

This process is finished when *all* points are visited.

The below picture ([source](https://medium.com/@elutins/dbscan-what-is-it-when-to-use-it-how-to-use-it-8bd506293818)) visually summarizes the difference between point types.

![](images/DBscan.jpg)

### Parameter Setting

Let's now try to model our goal in terms of parameters. `DBSCAN` is a nice technique for geo-referenced clustering because the `epsilon` parameter has a physical meaning. In this case, we can think of it as the drop-off area we deem is forming a cluster. 


This [website](https://www.calcmaps.com/map-radius/) comes in very handy to this end. With this tool, one can know the radius of a circle anywhere on Google Maps by simply clicking on a single point and extending or moving the circle to change the radius on the Map. 


![](images/radius_finder.png)



The ***arbitrarily*** drawn circle has a radius of 0.48 miles.

Further, considering that 1 degree latitude is $\approx$ 69 miles, we can find our `epsilon`:

$$ \frac{0.48 \ \ miles/radius}{69 \ \ miles/degree} \approx 0.007 $$ 

Finally, we may ***arbitrarily*** set the value of `MinPoints` to 500, but nothing would impede us from using, say, 1000.

In sum, the above reasoning leads to us the realization that: 

> <p><span style="color:red"><b>Clustering is Subjective!</b></span></p>

It helps us to analyze and get insight about the data, but the final result heavily depends on the user's choices.

### Model Training

One last step must be performed before running the algorithm, though. We need to convert our data into a format that is `DBSCAN`-friendly. To this end, we are required to convert the two columns in our `df_1` into an *numpy array*. 

In [31]:
# create a numpy array, each comprising both lat and long for the same point
lat_lon_points = df_1.to_numpy()

# print the first 10 entries
lat_lon_points[0:10]

array([[ 41.88099447, -87.63274649],
       [ 41.88099447, -87.63274649],
       [ 41.91461629, -87.63171737],
       [ 41.90749193, -87.63576009],
       [ 42.00157103, -87.69501259],
       [ 41.90585777, -87.63086503],
       [ 41.94651142, -87.80602   ],
       [ 41.88528132, -87.6572332 ],
       [ 41.9442266 , -87.65599818],
       [ 41.89204214, -87.63186395]])

Now, the parameters we have defined in the above section will be inputted in our model.

In [53]:
# create the clusters
cluster_scanned = DBSCAN(eps=0.007, min_samples=1000)

<div class="alert alert-block alert-success">
<b>Tip:</b> Run the below cell only if you are interested in trying the code out. Else, skip one cell and simply continue running the notebook from two cells below, thereby loading the corresponding saved model. 
</div>

In [54]:
# SKIP IF NOT INTERESTED IN TRYING THE CODE OUT

# allocate the drop-off points to each cluster
cluster_predicted = cluster_scanned.fit_predict(lat_lon_points)

In [45]:
# set a variable with the filename to be saved/loaded 
filename = './data/cluster_predicted.sav'

In [46]:
# save the model to disk
# pickle.dump(cluster_predicted, open(filename, 'wb'))

In [33]:
# load the model from disk
cluster_predicted = pickle.load(open(filename, 'rb'))

Now, we will allocate a label to each of the found clusters. A color will be **randomly** assigned too; thus, it should not surprise the fact that colors will almost surely be different every time the below cell is run.   

Further, the color ***WHITE*** is allocated to `outliers`. To avoid that another cluster - or more - could have it assigned too, the maximum possible value for the <span style="color:blue"><b>BLUE</b></span> is set to 200. In such a way, in the worst-case scenario the most "similar" color in terms of RGB would be a pale yellow. 

In [55]:
%matplotlib inline

# find the unique number of clusters
cluster_number = len(set(cluster_predicted))
print("{} clusters have been identified".format(cluster_number))

cluster_label = [str(cluster_predicted[label]) for label in cluster_predicted]

# create a tuple with RGB values for each cluster that was found
color_dict = {i: (randint(0, 255), randint(0, 255), randint(0, 200))
              for i in range(-1, cluster_number - 1)}

# assign WHITE to the outliers
color_dict[-1] = (255, 255, 255)

# create a list assigning the repsective color to each point based on cluster
cluster_color = [color_dict[i] for i in cluster_predicted]

# cluster_number_full = [str(i) for i in cluster_predicted]

27 clusters have been identified


To assess the goodness of the found clusters, then, we could plot our results on the map and compare them with the landmarks scraped from TripAdvisor.

Doing so, however, bears some drawbacks. Plotting about 200.000 points on the map is possible, of course, but would require a lot of computational power, thereby reducing the performance of our machine.

To alleviate this computational burden, we can leverage a peculiarity of our dataset, namely privacy. In fact, as the exact location down to the street address could affect privacy, Chicago's city hall decided to provide location only at the Census Tract and Community Area levels ([source](https://digital.chicago.gov/index.php/chicago-taxi-data-released/)).

This implies that two trips ending within the same Census Tract but at different locations will actually present the same GPS coordinates. In other words, the number of drop-off points is at least not larger to the number of Census Tracts, and therefore, plotting only one drop-off will suffice to represent its. 

To create a list of unique drop-off points, first, we will combine each pair of GPS coordinates with its cluster label.

In [56]:
# create a list containing tuples that merge latitutde, longitude and predicted cluster
lat_lon_points_merged = [(lat_lon[0], lat_lon[1], clus)
                         for lat_lon, clus in zip(lat_lon_points, cluster_predicted)]
lat_lon_points_merged[0:10]

[(41.880994471, -87.632746489, 0),
 (41.880994471, -87.632746489, 0),
 (41.914616286, -87.631717366, 1),
 (41.90749193, -87.63576009, 1),
 (42.001571027, -87.695012589, -1),
 (41.905857769, -87.63086502700001, 1),
 (41.94651142, -87.80602000200001, -1),
 (41.88528132, -87.65723320000001, 2),
 (41.944226601, -87.655998182, 1),
 (41.892042136, -87.63186395, 1)]

In [57]:
len(lat_lon_points_merged)

277897

Second, we will create a new list with only a unique entry per triplet: **(latitude, longitude, cluster label)**.

In [58]:
# shrink the list to only contain unique tuples
lat_lon_points_cleaned = list(dict.fromkeys(lat_lon_points_merged))
lat_lon_points_cleaned[0:10]

[(41.880994471, -87.632746489, 0),
 (41.914616286, -87.631717366, 1),
 (41.90749193, -87.63576009, 1),
 (42.001571027, -87.695012589, -1),
 (41.905857769, -87.63086502700001, 1),
 (41.94651142, -87.80602000200001, -1),
 (41.88528132, -87.65723320000001, 2),
 (41.944226601, -87.655998182, 1),
 (41.892042136, -87.63186395, 1),
 (41.891971508000005, -87.612945414, 1)]

Our list of observations to plot has shrunk from roughly 200.000 to 372.

In [59]:
# print the number of unique entries
len(lat_lon_points_cleaned)

388

Third, we will retrieve the RGB combination that we created for each cluster. This is necessary to plot each cluster with the correct color. 

In [60]:
# create a list with the color palette for each unique coordinate pair
cluster_color_unique = [color_dict[lat_lon_points_cleaned[i][2]]
                        for i in range(0, len(lat_lon_points_cleaned))]
cluster_color_unique[0:10]

[(73, 169, 45),
 (32, 133, 166),
 (32, 133, 166),
 (255, 255, 255),
 (32, 133, 166),
 (255, 255, 255),
 (22, 181, 152),
 (32, 133, 166),
 (32, 133, 166),
 (32, 133, 166)]

In [61]:
len(cluster_color_unique)

388

Fourth and last, we will create again a tuple containing GPS coordinates, but ***without*** the cluster label.

In [62]:
# create a new tuple which excludes the predicted cluster
lat_lon_points_unique = [(lat_lon_points_cleaned[i][0], lat_lon_points_cleaned[i][1])
                         for i in range(0, len(lat_lon_points_cleaned))]

### Model Evaluation

<div class="alert alert-block alert-success">
<b>Tip:</b> Run the code in this section only if you are interested in trying the code out. Else, its results are loaded as pictures. 
</div>

After the whole preparation, let's now plot our clusters together with the TripAdvisor's landamarks.

In [43]:
def cluster_plotter(core_only=False):
    """
    Function that plots points of interests and predicted clusters on Chicago's Google Map.abs

    Parameters:
    core_only (bool): whether to plot all points or only core ones (default False).

    """

    # create the map object
    map_cluster = gmaps.figure(map_type='ROADMAP', layout=figure_layout,
                               zoom_level=11, center=(41.834275, -87.691385))

    # generate the marker layer maps
    marker_layer = gmaps.marker_layer(
        points_of_interest_locations, info_box_content=points_of_interest_info)

    if core_only == False:

        # create the layer of ALL points to add to the map
        dropoff_layer = gmaps.symbol_layer(locations=lat_lon_points_unique, fill_color=cluster_color_unique,
                                           stroke_color=cluster_color_unique, scale=4)
    elif core_only == True:

        # create the layer of ONLY CORE points to add to the map
        dropoff_layer = gmaps.symbol_layer(locations=lat_lon_points_core_unique, fill_color=cluster_color_core_unique,
                                           stroke_color=cluster_color_core_unique, scale=4)

    # add the layer with PoI on the map
    map_cluster.add_layer(marker_layer)

    # add the layer with dropoffs on the map
    map_cluster.add_layer(dropoff_layer)

    # plot the map
    return map_cluster

In [64]:
# plot all points on the map
cluster_plotter(core_only=False)

![](images/cluster_500.jpg)

The map is rather crowded and honestly a bit confusing. The `outliers` - colored in ***white*** - are dominating the map points scattered all over the map.

A solution is to identify the so-called `core` points and plot them alone.

<div class="alert alert-block alert-danger">
<b>Attention:</b> Run the below cell only if you have performed `fit_predict()`. Else, skip one cell and simply continue running the notebook, thereby loading the saved array. 
</div>

In [48]:
# RUN ONLY IF TRYING THE CODE OUT
# find the index of the core points, thus removing outliers
cluster_core_idx = cluster_scanned.core_sample_indices_

# save the core index list as .txt
with open("./data/core_points.txt", "wb") as fp:
    pickle.dump(cluster_core_idx, fp)

In [49]:
# load the core index list
with open("./data/core_points.txt", "rb") as fp:
    cluster_core_idx = pickle.load(fp)

In [50]:
print("There are {} core points".format(len(cluster_core_idx)))

There are 266080 core points


In a similar vein of what we have done for the full points, we will now follow the same ideas, although quicker since the data has already been processed.

In [51]:
# extract the colors for each CORE point
cluster_color_core_unique = [color_dict[lat_lon_points_cleaned[i][2]] for i in range(
    0, len(lat_lon_points_cleaned)) if lat_lon_points_cleaned[i][2] != -1]

# create a list of tuples with GPS pairs of CORE points
lat_lon_points_core_unique = [(lat_lon_points_cleaned[i][0], lat_lon_points_cleaned[i][1])for i in range(
    0, len(lat_lon_points_cleaned)) if lat_lon_points_cleaned[i][2] != -1]

In [52]:
# plot the onyl the CORE points on the map
cluster_plotter(core_only=True)

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1200px'))

![](images/cluster_500_core_only.jpg)

Results appears encouraging. Considering our evaluation metric, there is a decent visual match between clusters and points of interests.

![](images/cluster_500_core_only_south.jpg)

Further, we extend our analysis to clusters that do not lie nearby points of interests.

For instance, it can be noted how the algorithm finds some clusters around the two major airports serving Chicago, namely *O'Hare International Airport* (top) and *Chicago Midway International Airport* (bottom).

![](images/cluster_500_core_only_airports.jpg)

The algorithm also correctly finds some clusters in the surrounding of natural points of interests, like beaches.

![](images/cluster_500_core_only_beaches.jpg)

Let's consider *Kathy Osterman beach. Despite being a truthful point of interest according to TripAdvisor, it has not been plotted as landmark given that its category "Beaches" was excluded from the parser function `landmarks_dict()`

![](images/kathy_beach.png)

However, these results also highlight the major limitations of `DBSCAN`. While it is great at separating high density clusters from low density clusters, DBSCAN struggles with clusters of similar density. 

The below picture summarizes it well. Observing at the spread of the <span style="color:silver"><b>silver</b></span> cluster, we could speculate that perhaps the algorithm has indeed encountered some difficulties in finding

![](images/cluster_500_core_only_limitations.jpg)

## Conclusions

In sum, the algorithm seems to perform well, overall. Despite the problem encountered   